##### @Time    : 2019/4/5 15:09
##### @Author  : ChanLiang
##### @Github  ：https://github.com/ChanLiang

# logistic regression 多分类（iris数据集）

In [27]:
import pandas as pd
import numpy as np
import numpy
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import array
import time
import scipy.sparse
import scipy.optimize

### 1. 制作数据集

训练集占70%，测试集占30%

In [38]:
iris = load_iris()
X, Y = iris.data, iris.target
#print (X)
#print (Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

### 2. 开始编写多分类器

In [47]:
class SoftmaxRegression(object):
    #######################################################################################
    """ 初始化分类器对象 """
    
    def __init__(self, input_size, num_classes, lamda):
    
        """ 初始化分类器对象的参数"""
    
        self.input_size  = input_size  # 输入向量的大小（一个输入实例x的特征数）
        self.num_classes = num_classes # 总共有多少个类
        self.lamda       = lamda       # 规则化系数
        
        """ 每个类都有自己的LR参数，将3个不同类的参数按行排列形成一个参数矩阵的'扁平形式':theta.shape = (3 * 4, 1) """
        
        rand = numpy.random.RandomState(int(time.time()))
        # theta.shape = (num_classes * input_size, 1)
        self.theta = 0.005 * numpy.asarray(rand.normal(size = (num_classes * input_size, 1)))
    
    #######################################################################################
    """ 返回一组输入数据的GroundTruth矩阵(压缩存储)：shape = (k x m)，每一列代表一个样本输入哪一个类(one-hot) """
        
    def getGroundTruth(self, labels):
    
        """ 准备构建 groundtruth matrix 的数据"""
    
        data   = numpy.ones(len(labels))
        indices = numpy.array(labels).flatten()  # 变成一维的
        indptr = numpy.arange(len(labels)+1)
        
        """ 构建 groundtruth matrix ，注意这里是按行构建的，需要转置一下再返回"""
        
        ground_truth = scipy.sparse.csr_matrix((data, indices, indptr))  # m x k
        ground_truth = numpy.transpose(ground_truth.todense())  # 转置成k x m
        
        return ground_truth
        
    #######################################################################################
    """ 返回代价函数的值，以及组合参数 theta 的梯度 """
        
    def softmaxCost(self, theta, input, labels):
    
        """ 构建 groundtruth matrix """
    
        ground_truth = self.getGroundTruth(labels)
        
        """ 调整 theta 的形状，以方便计算 """
        
        theta = theta.reshape(self.num_classes, self.input_size)
        
        """ 计算概率矩阵 probabilities matrix: (k, m)—— m个样本横向排列，每一列代表该样本属于不同类的概率 """
        
        theta_x       = numpy.dot(theta, input)
        hypothesis    = numpy.exp(theta_x)      
        probabilities = hypothesis / numpy.sum(hypothesis, axis = 0)
        
        """ 计算 经验风险 部分 """
        
        cost_examples    = numpy.multiply(ground_truth, numpy.log(probabilities))
        traditional_cost = -(numpy.sum(cost_examples) / input.shape[1])
        
        """ 计算 规则化项 部分：否则每当求得一个优化参数时，如果将这个参数的每一项都减掉同一个数，其得到的损失函数值也是一样的 """
        
        theta_squared = numpy.multiply(theta, theta)
        weight_decay  = 0.5 * self.lamda * numpy.sum(theta_squared)  # axis取None，即将数组/矩阵中的元素全部加起来，得到一个和
        
        """ 将 两部分 加起来，得到最终的 代价函数值 """
        
        cost = traditional_cost + weight_decay
        
        """ 计算 theta 的梯度 """
        
        theta_grad = -numpy.dot(ground_truth - probabilities, numpy.transpose(input))
        theta_grad = theta_grad / input.shape[1] + self.lamda * theta
        theta_grad = numpy.array(theta_grad)
        theta_grad = theta_grad.flatten()
        
        return [cost, theta_grad]
    
    #######################################################################################
    """ 对一组输入，返回他们对应的预测值 """
            
    def softmaxPredict(self, theta, input):
    
        """ 同上 """
        # K * m
        theta = theta.reshape(self.num_classes, self.input_size)
        
        """ 计算 概率矩阵 """
        
        theta_x       = numpy.dot(theta, input)
        hypothesis    = numpy.exp(theta_x)      
        probabilities = hypothesis / numpy.sum(hypothesis, axis = 0)
        
        """ 基于 概率矩阵 给出预测值 """
        
        predictions = numpy.zeros((input.shape[1], 1))
        predictions[:, 0] = numpy.argmax(probabilities, axis = 0)
        
        return predictions

In [48]:
""" 初始化数据, 用训练集训练模型 再该模型在测试集上进行预测 """

def executeSoftmaxRegression():
    
    """ 初始化分类器的参数 """
    
    input_size     = 4    # input vector size
    num_classes    = 3     # number of classes
    lamda          = 0.0001 # weight decay parameter
    max_iterations = 10    # number of optimization iterations
    
    """ 初始化训练数据 和 测试数据"""
    
    training_data = X_train.T
    training_labels = Y_train
    test_data  = X_test.T 
    test_labels = Y_test.reshape(len(Y_test), 1)
    
    """ 初始化一个分类器 """
    
    regressor = SoftmaxRegression(input_size, num_classes, lamda)
    
    """ 运行 L-BFGS 算法，获得最优的参数值 """
    
    opt_solution  = scipy.optimize.minimize(regressor.softmaxCost, regressor.theta, 
                                            args = (training_data, training_labels,), method = 'L-BFGS-B', 
                                            jac = True, options = {'maxiter': max_iterations})
    opt_theta     = opt_solution.x
    
    """ 用训练好的模型在测试集上做预测 """
    
    predictions = regressor.softmaxPredict(opt_theta, test_data)
    
    """ 打印测试集上的准确率 """
#     print (test_labels)
#     print (predictions)
    correct = test_labels[:, 0] == predictions[:, 0]
    print ("""Accuracy :""", numpy.mean(correct))
    
executeSoftmaxRegression()

Accuracy : 0.9777777777777777


### 结束